In [1]:
import pandas as pd
import os

# Helpful functions

In [2]:
def filter_for_our_years(df):
    return df[(df['yr_id'] == 2020) | (df['yr_id'] == 2021)]

# Age

In [3]:
def get_input_data(category, geo_level):
    # 2022_01
    df_2022_01 = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Data\{category}\{geo_level}_{category}_est_2022_01_ind_QA.csv')
    df_2022_01 = filter_for_our_years(df_2022_01)
    if geo_level == 'SRA':
        df_2022_01['SRA'] = df_2022_01['SRA'].str.lower()
    df_2022_01 = df_2022_01.set_index([geo_level, 'yr_id'])
    
    if category == 'income_group':
        category = 'household_income'
    if geo_level == 'SRA':
        geo_level = 'sra'
    # 2021_01 
    df_2021_01 = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\{category}\QA_2021_01_{geo_level}_{category}.csv')
    if geo_level == 'sra':
        df_2021_01['SRA'] = df_2021_01['sra']
        df_2021_01 = df_2021_01.drop('sra', axis=1)
        geo_level = 'SRA'
        df_2021_01['SRA'] = df_2021_01['SRA'].str.lower()
    if category == 'age_sex_ethnicity':
        df_2021_01 = df_2021_01.rename(columns={'name':'age_group'})
    
    df_2021_01 = filter_for_our_years(df_2021_01)
    df_2021_01 = df_2021_01.set_index([geo_level, 'yr_id'])
    
    return df_2022_01, df_2021_01

In [4]:
def get_input_data_combo_files(geo_level):
    # 2022_01
    df_2022_01 = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Data\combo_files\{geo_level}_housing_population_households_combo_QA.csv')
    df_2022_01 = filter_for_our_years(df_2022_01)
    df_2022_01[geo_level] = df_2022_01[geo_level].str.lower()
    df_2022_01 = df_2022_01.set_index([geo_level, 'yr_id'])
    #print('good')

    # 2021_01 
    if geo_level == 'SRA':
        geo_level = 'sra'
    df_2021_01 = pd.read_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\combo_files\2021_01_{geo_level}_housing_population_households_combo_QA.csv')
    #print('good')
    df_2021_01 = filter_for_our_years(df_2021_01)
    #print(df_2021_01.columns)
    if geo_level == 'sra':
        df_2021_01['SRA'] = df_2021_01['sra']
        df_2021_01 = df_2021_01.drop('sra', axis=1)
        geo_level = 'SRA'
    df_2021_01[geo_level] = df_2021_01[geo_level].str.lower()
    df_2021_01 = df_2021_01.set_index([geo_level, 'yr_id'])
    
    return df_2022_01, df_2021_01

# Functions

In [10]:
# import pandas as pd

# def subtract_dataframes(df1, df2):
#     # Find the common columns between the two data frames
#     common_cols = list(set(df1.columns) & set(df2.columns))
#     common_cols = [x for x in df2.columns if x in common_cols]
#     # Filter the data frames to keep only the common columns
#     df1 = df1[common_cols]
#     df2 = df2[common_cols]
    
#     # Subtract the second data frame from the first
#     result = df1.subtract(df2)
    
#     return result


In [7]:
# def subtract_dataframes(df1, df2):
#     # Filter the second data frame to keep only rows with index values in the first data frame
#     df2 = df2.loc[df1.index]
    
#     # Find the common columns between the two data frames
#     common_cols = list(set(df1.columns) & set(df2.columns))
#     common_cols = [x for x in df2.columns if x in common_cols]
    
#     # Filter the data frames to keep only the common columns
#     df1 = df1[common_cols]
#     df2 = df2[common_cols]
    
#     # Subtract the second data frame from the first
#     result = df1.subtract(df2)
    
#     return result


In [5]:
def subtract_dataframes(df1, df2):
    # Filter the second data frame to keep only rows with index values in the first data frame
    common_index = pd.merge(df1, df2, left_index=True, right_index=True)
    df1 = df1.loc[common_index.index]
    df2 = df2.loc[common_index.index]
    
    # Find the common columns between the two data frames
    common_cols = list(set(df1.columns) & set(df2.columns))
    common_cols = [x for x in df2.columns if x in common_cols]
    
    # Filter the data frames to keep only the common columns
    df1 = df1[common_cols]
    df2 = df2[common_cols]
    
    # Subtract the second data frame from the first
    result = df1.subtract(df2)
    
    return result

In [6]:
def write_to_excel(df_2022, df_2021, category, geo_level):
    diff = subtract_dataframes(df_2022, df_2021)
    # Create a Pandas Excel writer using xlsxwriter as the engine
    writer = pd.ExcelWriter(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\diff_outputs\{category}\{geo_level}_diff_2022_minus_2021.xlsx', engine='xlsxwriter')

    # Write each data frame to a different sheet
    df_2022.reset_index().to_excel(writer, sheet_name='2022_01 Data', index=False)
    df_2021.reset_index().to_excel(writer, sheet_name='2021_01 Data', index=False)
    diff.reset_index().to_excel(writer, sheet_name='Diff', index=False)

    # Save the writer
    writer.save()

In [7]:
def write_to_excel_combo_files(df_2022, df_2021, geo_level):
    diff = subtract_dataframes(df_2022, df_2021)
    # Create a Pandas Excel writer using xlsxwriter as the engine
    writer = pd.ExcelWriter(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\diff_outputs\combo_files\{geo_level}_diff_2022_minus_2021.xlsx', engine='xlsxwriter')

    # Write each data frame to a different sheet
    df_2022.reset_index().to_excel(writer, sheet_name='2022_01 Data', index=False)
    df_2021.reset_index().to_excel(writer, sheet_name='2021_01 Data', index=False)
    diff.reset_index().to_excel(writer, sheet_name='Diff', index=False)

    # Save the writer
    writer.save()

# Run

In [8]:
categories = ['age', 'ethnicity', 'households', 'housing', 'income_group', 'population', 'sex']
geo_levels = ['SRA', 'cpa', 'jurisdiction', 'region']

In [9]:
for category in categories:
    for geo_level in geo_levels:
        df_2022_01, df_2021_01 = get_input_data(category=category, geo_level=geo_level)
        write_to_excel(df_2022_01, df_2021_01, category = category, geo_level = geo_level)
        print(f'{category}-{geo_level} is complete')

age-SRA is complete
age-cpa is complete
age-jurisdiction is complete
age-region is complete
ethnicity-SRA is complete
ethnicity-cpa is complete
ethnicity-jurisdiction is complete
ethnicity-region is complete
households-SRA is complete
households-cpa is complete
households-jurisdiction is complete
households-region is complete
housing-SRA is complete
housing-cpa is complete
housing-jurisdiction is complete
housing-region is complete
income_group-SRA is complete
income_group-cpa is complete
income_group-jurisdiction is complete
income_group-region is complete
population-SRA is complete
population-cpa is complete
population-jurisdiction is complete
population-region is complete
sex-SRA is complete
sex-cpa is complete
sex-jurisdiction is complete
sex-region is complete


In [19]:
import numpy as np
for geo_level in geo_levels:
    df_2022_01, df_2021_01 = get_input_data_combo_files(geo_level)
    if geo_level == 'SRA':
        print(len(df_2021_01))
        df_2021_01 = df_2021_01.reset_index()
        df_2021_01['SRA'] = np.where(df_2021_01['SRA'] == 'harbison-crest', 'harbison crest', df_2021_01['SRA'])
        df_2021_01 = df_2021_01.set_index(['SRA', 'yr_id'])
    write_to_excel_combo_files(df_2022_01, df_2021_01, geo_level)
    print(f'{geo_level}-is complete')

82
SRA-is complete
cpa-is complete
jurisdiction-is complete
region-is complete


In [130]:
category = 'age_sex_ethnicity'
for geo_level in geo_levels:
    df_2022_01, df_2021_01 = get_input_data(category=category, geo_level=geo_level)
    df_2022_01 = df_2022_01.reset_index().groupby([geo_level, 'yr_id', 'age_group', 'sex']).sum()
    df_2021_01 = df_2021_01.reset_index().groupby([geo_level, 'yr_id', 'age_group', 'sex']).sum()
    write_to_excel(df_2022_01, df_2021_01, category = category, geo_level = geo_level)
    print(f'{geo_level}-is complete')

SRA-is complete
cpa-is complete
jurisdiction-is complete
region-is complete


In [122]:
category = 'age_sex_ethnicity'
geo_level = 'region'

In [123]:
df_2022_01, df_2021_01 = get_input_data(category=category, geo_level=geo_level)


In [124]:
df_2022_01 = df_2022_01.reset_index().groupby([geo_level, 'yr_id', 'age_group', 'sex']).sum()
df_2021_01 = df_2021_01.reset_index().groupby([geo_level, 'yr_id', 'age_group', 'sex']).sum()

In [125]:
df_2021_01 = df_2021_01.reset_index().groupby([geo_level, 'yr_id', 'age_group', 'sex']).sum()

In [126]:
write_to_excel(df_2022_01, df_2021_01, category = category, geo_level = geo_level)

In [ ]:
# Add name and sex in the groupby in data input